Tahap 6: Automation Logic & Alert System
1. Research Objectives
Mengintegrasikan model prediksi ke dalam fungsi otomasi yang siap digunakan (Production-ready).

Membangun logika pemicu (Trigger Logic) untuk sistem peringatan risiko dini.

Membuat sistem pelaporan otomatis untuk memandu tindakan tim Manajemen Risiko.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# 1. Tentukan path file
file_path = "../data/processed/personal_finance_with_risk_scores.csv"

# 2. Cek apakah file ada sebelum di-load (Proteksi Profesional)
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    
    # Bersihkan nama kolom dari spasi tersembunyi
    df.columns = df.columns.str.strip()
    
    print(f"✅ Sistem siap. Berhasil memuat data dari {len(df)} nasabah.")
    print(f"Kolom tersedia: {df.columns.tolist()[:5]}... (dan lainnya)")
else:
    print(f"❌ ERROR: File '{file_path}' tidak ditemukan!")
    print("Saran: Jalankan cell export di akhir Notebook Tahap 05 terlebih dahulu.")

# Tampilkan 5 data teratas untuk memastikan kolom risk_score & risk_category sudah ada
if 'risk_score' in df.columns:
    display(df[['monthly_income_usd', 'risk_score', 'risk_category']].head())

✅ Sistem siap. Berhasil memuat data dari 32424 nasabah.
Kolom tersedia: ['user_id', 'age', 'gender', 'education_level', 'employment_status']... (dan lainnya)


,monthly_income_usd,risk_score,risk_category
0,3531.69,80.24,Low Risk
1,3531.73,43.55,Medium Risk
2,2799.49,95.48,Elite
3,5894.88,56.49,Medium Risk
4,5128.93,84.28,Low Risk


In [3]:
def automated_credit_decision(customer_data):
    """
    Logika Otomasi Keputusan Kredit:
    - Elite: Auto-Approve + Platinum Card Offer
    - Low Risk: Auto-Approve + Standard Offer
    - Medium Risk: Referral to Manual Review
    - High Risk: Auto-Reject + Financial Counseling Offer
    """
    risk_cat = customer_data['risk_category']
    score = customer_data['risk_score']
    
    if risk_cat == 'Elite':
        decision = "✅ AUTO-APPROVE (Premium)"
        action = "Kirim penawaran Kartu Kredit Platinum & Limit Tinggi."
    elif risk_cat == 'Low Risk':
        decision = "✅ AUTO-APPROVE (Standard)"
        action = "Kirim syarat dan ketentuan pinjaman standar."
    elif risk_cat == 'Medium Risk':
        decision = "⚠️ MANUAL REVIEW REQUIRED"
        action = "Teruskan ke tim analis untuk verifikasi dokumen tambahan."
    else: # High Risk
        decision = "❌ AUTO-REJECT"
        action = "Kirim saran program restrukturisasi atau konseling keuangan."
        
    return decision, action

# Simulasi pada 5 nasabah pertama
sample_decisions = df.head(5).apply(lambda x: automated_credit_decision(x), axis=1)
df_sample = df.head(5).copy()
df_sample['System_Decision'], df_sample['Next_Action'] = zip(*sample_decisions)

display(df_sample[['risk_score', 'risk_category', 'System_Decision', 'Next_Action']])

,risk_score,risk_category,System_Decision,Next_Action
0,80.24,Low Risk,✅ AUTO-APPROVE (Standard),Kirim syarat dan ketentuan pinjaman standar.
1,43.55,Medium Risk,⚠️ MANUAL REVIEW REQUIRED,Teruskan ke tim analis untuk verifikasi dokume...
2,95.48,Elite,✅ AUTO-APPROVE (Premium),Kirim penawaran Kartu Kredit Platinum & Limit ...
3,56.49,Medium Risk,⚠️ MANUAL REVIEW REQUIRED,Teruskan ke tim analis untuk verifikasi dokume...
4,84.28,Low Risk,✅ AUTO-APPROVE (Standard),Kirim syarat dan ketentuan pinjaman standar.


In [4]:
def risk_alert_trigger(df):
    """
    Mendeteksi nasabah yang memenuhi kriteria 'Red Flag':
    1. DTI > 0.6 (Hutang lebih dari 60% pendapatan)
    2. Risk Score < 30
    3. Credit Score < 500
    """
    red_flags = df[
        (df['debt_to_income_ratio'] > 0.6) | 
        (df['risk_score'] < 30) | 
        (df['credit_score'] < 500)
    ].copy()
    
    red_flags['Alert_Timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    red_flags['Alert_Priority'] = 'CRITICAL'
    
    return red_flags

# Menjalankan Sistem Peringatan
critical_alerts = risk_alert_trigger(df)

print(f"🚨 ALERT SYSTEM: Ditemukan {len(critical_alerts)} nasabah dengan kriteria risiko kritis!")
display(critical_alerts[['risk_score', 'debt_to_income_ratio', 'Alert_Priority', 'Alert_Timestamp']].head())

🚨 ALERT SYSTEM: Ditemukan 17904 nasabah dengan kriteria risiko kritis!


,risk_score,debt_to_income_ratio,Alert_Priority,Alert_Timestamp
0,80.24,0.00,CRITICAL,2026-01-08 17:56:47
1,43.55,1.40,CRITICAL,2026-01-08 17:56:47
3,56.49,0.84,CRITICAL,2026-01-08 17:56:47
5,-189.11,9.43,CRITICAL,2026-01-08 17:56:47
6,-110.89,6.89,CRITICAL,2026-01-08 17:56:47


In [5]:
# Membuat Daily Action List
action_list = df.copy()
action_list['Decision'], action_list['Action'] = zip(*action_list.apply(automated_credit_decision, axis=1))

# Export data untuk Dashboard/Operasional
action_list.to_csv("../data/processed/daily_risk_actions.csv", index=False)
critical_alerts.to_csv("../data/processed/critical_alerts_log.csv", index=False)

print("📂 Laporan harian dan log peringatan telah diekspor ke folder /processed/.")

📂 Laporan harian dan log peringatan telah diekspor ke folder /processed/.


In [6]:
def validate_automation_data(df):
    """
    Memastikan data layak untuk diproses otomasi.
    - Cek nilai null pada kolom krusial.
    - Cek nilai negatif yang tidak masuk akal pada income.
    """
    essential_cols = ['risk_score', 'debt_to_income_ratio', 'credit_score']
    
    # Check for missing values
    missing_data = df[essential_cols].isnull().sum()
    
    # Check for logical errors
    logical_errors = df[df['monthly_income_usd'] < 0]
    
    if missing_data.sum() == 0 and len(logical_errors) == 0:
        print("✅ Data Validation Passed: Data aman untuk diproses otomasi.")
        return True
    else:
        print("⚠️ Data Validation Failed: Ditemukan ketidakkonsistenan data.")
        return False

# Jalankan validasi sebelum otomasi
validate_automation_data(df)

✅ Data Validation Passed: Data aman untuk diproses otomasi.


True

In [7]:
# Menghitung metrik efisiensi otomasi
total_customers = len(action_list)
auto_processed = action_list[action_list['Decision'].str.contains('AUTO')].shape[0]
manual_review = action_list[action_list['Decision'].str.contains('MANUAL')].shape[0]

efficiency_rate = (auto_processed / total_customers) * 100

print(f"📊 Automation Efficiency: {efficiency_rate:.2f}%")
print(f"🤖 Keputusan Otomatis: {auto_processed}")
print(f"👤 Perlu Review Manual: {manual_review}")

📊 Automation Efficiency: 89.67%
🤖 Keputusan Otomatis: 29075
👤 Perlu Review Manual: 3349


📊 Summary of Insights & Strategy (Stage 06 - Final Version)
1. Operational Excellence & Efficiency ⚙️
Scalable Decision Making: Sistem otomasi berhasil mengambil alih lebih dari 70% beban kerja persetujuan kredit melalui fungsi automated_credit_decision, memungkinkan tim analis fokus pada kasus kompleks.

Consistent Governance: Dengan logika berbasis kode, bank memastikan setiap nasabah dinilai secara objektif berdasarkan risiko, mengurangi risiko bias manusia dalam pemberian pinjaman.

2. Real-Time Risk Surveillance 🚨
Zero-Latency Alerts: Sistem risk_alert_trigger mampu menyaring nasabah dengan profil kritis (seperti DTI > 60%) secara instan, memberikan perlindungan likuiditas bank secara real-time.

Audit Trail: Penggunaan Alert_Timestamp memastikan setiap peringatan terekam secara kronologis, memudahkan audit internal jika terjadi lonjakan kredit macet di masa depan.

3. Strategic Business Logic 📈
Tiered Response Strategy: Strategi respons berjenjang—dari Auto-Approve hingga Financial Counseling—memastikan bank tetap menjaga hubungan baik bahkan dengan nasabah berisiko tinggi melalui pendekatan solutif.

Resource Optimization: Dengan memisahkan nasabah "Elite" dan "High Risk", bank dapat mengalokasikan sumber daya pemasaran dan penagihan dengan jauh lebih akurat.

🚀 Next Step: Tahap 07 - Final Insight & Business Impact
Ini adalah tahap akhir di mana kita akan menyusun narasi kemenangan untuk para Stakeholders:

💰 ROI & Loss Prevention Analysis: Mengestimasi berapa banyak potensi kerugian yang bisa dihindari berkat sistem peringatan dini di Tahap 06.

📊 Executive Dashboard: Membuat ringkasan visual dari seluruh perjalanan data (Tahap 1-6) untuk level Direksi.

🗺️ Product Development Roadmap: Memberikan rekomendasi strategis untuk produk keuangan masa depan berdasarkan profil nasabah yang paling menguntungkan.